In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['dta3.csv']


In [2]:
dt=pd.read_csv('../input/dta3.csv')


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dt=pd.get_dummies(dt)

In [4]:
dt=dt.drop(columns=['B_ELECTRONIC_AUCTION_N','PROCEDURE_RES','SECTOR_na','B_GPA_N','TYPE_OF_CONTRACT_W','CAE_TYPE_Z','AUTHORITY_COUNTRY_UK'] )

In [5]:
print(dt.columns)
len(dt)

Index(['YEAR', 'NUMBER_OFFERS', 'AWARD_EST_VALUE_EURO', 'AWARD_VALUE_EURO',
       'AUTHORITY_COUNTRY_AT', 'AUTHORITY_COUNTRY_BE', 'AUTHORITY_COUNTRY_BG',
       'AUTHORITY_COUNTRY_CH', 'AUTHORITY_COUNTRY_CY', 'AUTHORITY_COUNTRY_CZ',
       ...
       'SECTOR_93', 'SECTOR_98', 'B_GPA_Y', 'PROCEDURE_AWP', 'PROCEDURE_COD',
       'PROCEDURE_INP', 'PROCEDURE_NIC', 'PROCEDURE_NOC', 'PROCEDURE_OPE',
       'B_ELECTRONIC_AUCTION_Y'],
      dtype='object', length=145)


2875612

In [6]:
dt = dt[pd.notnull(dt['AWARD_VALUE_EURO'])]
dt = dt[dt['AWARD_VALUE_EURO'] != 0]
len(dt)



2875612

In [7]:
dt = dt[pd.notnull(dt['AWARD_EST_VALUE_EURO'])]
dt = dt[dt['AWARD_EST_VALUE_EURO'] != 0]
len(dt)



1476395

In [8]:
dt = dt[pd.notnull(dt['NUMBER_OFFERS'])]
dt = dt[dt['NUMBER_OFFERS'] != 0]
len(dt)

1433056

In [9]:
import numpy as np
dt['LNRATIO']=(np.log(dt['AWARD_VALUE_EURO']/dt['AWARD_EST_VALUE_EURO']))
len(dt)


1433056

In [10]:
#log of 'df_train['month'] = pd.to_datetime(df_train.Date).dt.month
dt['LN_AWARD_VALUE_EURO'] = np.log(dt['AWARD_VALUE_EURO'])
dt['LN_AWARD_EST_VALUE_EURO'] = np.log(dt['AWARD_EST_VALUE_EURO'])

In [11]:
dt2=dt

In [12]:
dt = dt[(dt.LNRATIO>-1.386) & (dt.LNRATIO<0.642)]


In [13]:
train=dt[(dt.YEAR<2017)]
test=dt[(dt.YEAR==2017)]

In [14]:
y_train = train['LNRATIO']
X_train = train.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO', 'LNRATIO', 'LN_AWARD_VALUE_EURO'], axis=1)
y_test = test['LNRATIO']
X_test = test.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO',  'LNRATIO' ,'LN_AWARD_VALUE_EURO'], axis=1)


In [15]:
import math
from scipy.stats import spearmanr
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    print('Train rmse '+str(rmse(m.predict(X_train), y_train))) 
    print('Test  rmse '+str(rmse(m.predict(X_test), y_test)))
    print('Train '+str(spearmanr(m.predict(X_train),y_train)))
    print('Test  '+str(spearmanr(m.predict(X_test),y_test)))

In [16]:
import numpy as np
import pandas as pd

from hyperopt import fmin, hp, tpe, Trials, space_eval
from hyperopt.fmin import fmin

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

import xgboost as xgb

import lightgbm as lgbm

In [17]:
import catboost as cb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
def obj(params,X,y):
    params = {
        'depth': int(params['depth']),
        'iterations': int(params['iterations']) 
    }
    
    clf = cb.CatBoostRegressor(
       
        **params
    )
    
    score = cross_val_score(clf, X, y, scoring='neg_mean_squared_error',cv=2).mean()
    print("Score {:.4f} params {}".format(score, params))
    return score
objective = lambda params: (-1)*obj(params, X_train, y_train)
spCB = {
    'depth': hp.quniform('depth', 2, 8, 1),
    'iterations': hp.choice('iterations', [200,500,1000])
}
bestCB = fmin(fn=objective,
            space=spCB,
            algo=tpe.suggest,
            max_evals=50)



0:	learn: 0.3181081	total: 158ms	remaining: 31.4s

1:	learn: 0.3157256	total: 241ms	remaining: 23.8s

2:	learn: 0.3134707	total: 322ms	remaining: 21.1s

3:	learn: 0.3113348	total: 407ms	remaining: 19.9s

4:	learn: 0.3093072	total: 491ms	remaining: 19.2s

5:	learn: 0.3073787	total: 572ms	remaining: 18.5s

6:	learn: 0.3055612	total: 658ms	remaining: 18.1s

7:	learn: 0.3038403	total: 747ms	remaining: 17.9s

8:	learn: 0.3022143	total: 832ms	remaining: 17.7s

9:	learn: 0.3006705	total: 921ms	remaining: 17.5s

10:	learn: 0.2990425	total: 1.01s	remaining: 17.4s

11:	learn: 0.2975026	total: 1.1s	remaining: 17.3s

12:	learn: 0.2961825	total: 1.19s	remaining: 17s

13:	learn: 0.2949175	total: 1.27s	remaining: 16.9s

14:	learn: 0.2935916	total: 1.36s	remaining: 16.8s

15:	learn: 0.2924818	total: 1.44s	remaining: 16.6s

16:	learn: 0.2914435	total: 1.53s	remaining: 16.4s

17:	learn: 0.2902947	total: 1.61s	remaining: 16.3s

18:	learn: 0.2892088	total: 1.7s	remaining: 16.2s

19:	learn: 0.2882886	total

In [18]:
bestCBP = space_eval(spCB, bestCB)
print("BEST PARAMETERS: " + str(bestCBP))

BEST PARAMETERS: {'depth': 8.0, 'iterations': 1000}


In [19]:
from sklearn.metrics import r2_score
est = cb.CatBoostRegressor(**bestCBP) 
# Fit    
est.fit(X_train, y_train)
y_pred = est.predict(X_test)
 
# Predict
score = rmse(y_test, y_pred)

print("RMSE SCORE ON TEST DATA: {}".format(score))
r2a = r2_score(y_test, y_pred)
print("RMSE SCORE ON TEST DATA: {}".format(score))

0:	learn: 0.3242028	total: 278ms	remaining: 4m 37s
1:	learn: 0.3215801	total: 490ms	remaining: 4m 4s
2:	learn: 0.3190401	total: 705ms	remaining: 3m 54s
3:	learn: 0.3166485	total: 919ms	remaining: 3m 48s
4:	learn: 0.3143726	total: 1.14s	remaining: 3m 46s
5:	learn: 0.3122862	total: 1.35s	remaining: 3m 43s
6:	learn: 0.3102945	total: 1.57s	remaining: 3m 42s
7:	learn: 0.3083799	total: 1.78s	remaining: 3m 40s
8:	learn: 0.3065186	total: 1.99s	remaining: 3m 39s
9:	learn: 0.3048109	total: 2.21s	remaining: 3m 38s
10:	learn: 0.3032269	total: 2.41s	remaining: 3m 36s
11:	learn: 0.3017176	total: 2.61s	remaining: 3m 35s
12:	learn: 0.3002597	total: 2.83s	remaining: 3m 35s
13:	learn: 0.2988073	total: 3.06s	remaining: 3m 35s
14:	learn: 0.2975108	total: 3.27s	remaining: 3m 34s
15:	learn: 0.2962963	total: 3.47s	remaining: 3m 33s
16:	learn: 0.2951153	total: 3.69s	remaining: 3m 33s
17:	learn: 0.2940021	total: 3.9s	remaining: 3m 32s
18:	learn: 0.2929681	total: 4.09s	remaining: 3m 31s
19:	learn: 0.2920012	tot